In [1]:
from py_eddy_tracker import data
from py_eddy_tracker.dataset.grid import RegularGridDataset
from py_eddy_tracker.observations.observation import EddiesObservations

import os, glob
from datetime import datetime
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
from numpy import arange
import xarray as xr
from netCDF4 import Dataset


### Functions to invoke PyEddyTracker

###### https://py-eddy-tracker.readthedocs.io/en/stable/_autosummary/py_eddy_tracker.dataset.grid.RegularGridDataset.html?highlight=bessel_high_filter#py_eddy_tracker.dataset.grid.RegularGridDataset.bessel_high_filter
###### https://py-eddy-tracker.readthedocs.io/en/stable/_autosummary/py_eddy_tracker.appli.grid.identification.html?highlight=eddy_identification#py_eddy_tracker.appli.grid.identification

In [13]:
# Function to convert longitude from 0..360 to -180..180

def modify_lon(eddy_file):
    foo =  Dataset(eddy_file, "a")
    foo.variables['longitude'][:] = foo.variables['longitude'][:] - 360
    foo.variables['longitude_max'][:] = foo.variables['longitude_max'][:] - 360
    foo.variables['effective_contour_longitude'][:] = foo.variables['effective_contour_longitude'][:] -360
    foo.variables['speed_contour_longitude'][:] = foo.variables['speed_contour_longitude'][:] - 360
    foo.close
    
def PyEddy(hycom_file, directory_out, pre_filter = False, plot_png = False):

    # Load hycom data (adt = ssh, ugos, vgos)
    hycom = RegularGridDataset(hycom_file, 'lon', 'lat')
    
    # Extract Date from file
    date = os.path.basename(hycom_file)[15:23]
    print(date, int(date[0:4]), int(date[4:6]), int(date[6:8]))
    
    if pre_filter:
        hycom.bessel_high_filter('adt', 500, order=3)  # 500 km
        
    a, c = hycom.eddy_identification(
        'adt', 'ugos', 'vgos',    # Variables used for identification
        datetime(int(date[0:4]), int(date[4:6]), int(date[6:8])),
        0.005,                    # step between two isolines of detection (m)
        pixel_limit=(100, 200000),  # Min and max pixel count for valid contour
        shape_error=55)           # Error max (%) between ratio of circle fit and contour

    print(type(a))
    with Dataset(directory_out + '/Anticyclonic_' + date + '.nc', 'w') as h:
        a.to_netcdf(h)
    with Dataset(directory_out + '/Cyclonic_' + date + '.nc', 'w') as h:
        c.to_netcdf(h)

    modify_lon(directory_out + '/Anticyclonic_' + date + '.nc') 
    modify_lon(directory_out + '/Cyclonic_' + date + '.nc')  
    
    if plot_png:
        
        # Read in lon corrected eddy files
        a = EddiesObservations.load_file(directory_out + '/Anticyclonic_' + date + '.nc')
        c = EddiesObservations.load_file(directory_out + '/Cyclonic_' + date + '.nc')
        
        fig = plt.figure(figsize=(8, 6), dpi=80)
        ax = fig.add_axes([.03,.03,.94,.94])
        hycom.display(ax, 'adt', vmin=-0.3, vmax=0.3, cmap='viridis')
        a.display(ax, color="r", lw=2) 
        c.display(ax, color="w", lw=2)
        
        plt.savefig(directory_out + '/Eddies_' + date + '.png')

### Loop through Files (example is year 2003)

In [ ]:
hycom_in = sorted(glob.glob('/media/gsilsbe/modis1/gom/hycom/aviso_in/hycom_gomu_501_2003*.nc'))
directory_out = 'other'

for i in hycom_in:
    PyEddy(i, directory_out, pre_filter = True, plot_png = True)


We assume pixel position of grid is centered for /media/gsilsbe/modis1/gom/hycom/aviso_in/hycom_gomu_501_2003010100_t000.nc


20030101 2003 1 1
<class 'py_eddy_tracker.observations.observation.EddiesObservations'>


We assume pixel position of grid is centered for /media/gsilsbe/modis1/gom/hycom/aviso_in/hycom_gomu_501_2003010200_t000.nc


20030102 2003 1 2
<class 'py_eddy_tracker.observations.observation.EddiesObservations'>


We assume pixel position of grid is centered for /media/gsilsbe/modis1/gom/hycom/aviso_in/hycom_gomu_501_2003010300_t000.nc


...

20030107 2003 1 7
<class 'py_eddy_tracker.observations.observation.EddiesObservations'>


We assume pixel position of grid is centered for /media/gsilsbe/modis1/gom/hycom/aviso_in/hycom_gomu_501_2003010800_t000.nc


20030108 2003 1 8
<class 'py_eddy_tracker.observations.observation.EddiesObservations'>


We assume pixel position of grid is centered for /media/gsilsbe/modis1/gom/hycom/aviso_in/hycom_gomu_501_2003010900_t000.nc


20030109 2003 1 9
